<a href="https://colab.research.google.com/github/jaekyoungkim/NLP_STUDY/blob/main/BERT_%EB%AA%A8%EB%8D%B8_%EC%83%98%ED%94%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [BERT 모델을 이용해서  학습하기]  

### 1. 패키지 설치 
https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/ 참고

In [16]:
!pip install pytorch-transformers

In [17]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [18]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 21.99 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


### 2. 데이터 불러오기

In [19]:
train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')

In [22]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [24]:
test_df.head(3)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0


In [28]:
train_df.shape

(150000, 3)

In [29]:
test_df.shape

(50000, 3)

In [30]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=0.4, random_state=999)
test_df = test_df.sample(frac=0.4, random_state=999)

In [31]:
train_df.shape

(59998, 3)

In [33]:
test_df.shape

(19999, 3)

In [35]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [36]:
nsmc_train_dataset = NsmcDataset(train_df)


In [37]:
nsmc_train_dataset

In [38]:
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)
# Batch size는 2로 하였는데, 4부터는 Colab의 CUDA 메모리가 터지기 때문이다.. 때문에 굉장히 오래걸리고 NLP가 자원싸움

### 3. 학습

In [39]:
device = torch.device("cuda")
# gpu사용으로 변경

In [ ]:
# from_pretrained(‘bert-base-multilingual-cased’)를 사용함으로써 google에서 pretrained한 모델을 사용할 수 있다.
# bert 소개글에서와 같이 tokenizer는 wordpiece를 만들어 토큰화가 이루어진다. 
# BertForSequenceClassification모델을 사용하는 것이다. 
# 이 모델은 디폴트로 이진분류가 되어있다. 
# 따라서 multi-label을 하거나 다른 task를 학습하기 위해서는 구현된 다른 서브 모델들을 활용하면 된다.
# multi-label을 하고 싶으면 기본 BertModel의 classifier layer를 조정해줘야함

# https://ichi.pro/ko/hugging-face-byeonhwangi-laibeuleolieseo-bertleul-sayonghaneun-bangbeob-232493595427337 참고ㄲ

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

In [41]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0


model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [42]:
for epoch in range(epochs): # 위에서 정한 에폭만큼 학습
    
    for text, label in train_loader: # train loader에서 text 와 label컬럼 꺼내기
        optimizer.zero_grad()  # 
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]  # 
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list] # 512까지 공간의 빈부분을 0으로 패딩
        
        sample = torch.tensor(padded_list) # 텐서로 변환
        sample, label = sample.to(device), label.to(device) # gpu상에서 돌리고자하는거
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels) # 모델에 데이터를 넣어서 
        loss, logits = outputs  # 모델을 돌려서 나온 output을 loss, logits에 할당함

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)  # 맞은것의 개수 . eq 명령어 활용
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:   # itr주기마다 아래와 같은 문자 출력해줌
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


[Epoch 1/1] Iteration 500 -> Train Loss: 0.6936, Accuracy: 0.520
[Epoch 1/1] Iteration 1000 -> Train Loss: 0.6893, Accuracy: 0.544
[Epoch 1/1] Iteration 1500 -> Train Loss: 0.6976, Accuracy: 0.520
[Epoch 1/1] Iteration 2000 -> Train Loss: 0.6958, Accuracy: 0.504
[Epoch 1/1] Iteration 2500 -> Train Loss: 0.6911, Accuracy: 0.526
[Epoch 1/1] Iteration 3000 -> Train Loss: 0.6957, Accuracy: 0.499
[Epoch 1/1] Iteration 3500 -> Train Loss: 0.6901, Accuracy: 0.534
[Epoch 1/1] Iteration 4000 -> Train Loss: 0.6927, Accuracy: 0.540
[Epoch 1/1] Iteration 4500 -> Train Loss: 0.6852, Accuracy: 0.544
[Epoch 1/1] Iteration 5000 -> Train Loss: 0.6676, Accuracy: 0.603
[Epoch 1/1] Iteration 5500 -> Train Loss: 0.6127, Accuracy: 0.675
[Epoch 1/1] Iteration 6000 -> Train Loss: 0.6142, Accuracy: 0.672
[Epoch 1/1] Iteration 6500 -> Train Loss: 0.5815, Accuracy: 0.700
[Epoch 1/1] Iteration 7000 -> Train Loss: 0.5443, Accuracy: 0.733
[Epoch 1/1] Iteration 7500 -> Train Loss: 0.5447, Accuracy: 0.718
[Epoch 1/1]

### 4. 평가 

In [43]:
# evaluation
model.eval()
# 모델의 구성을 보여줌

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [44]:
# 테스트 데이터 dataloader에 탑재
nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)


In [45]:

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)  # 앞에서 학습한 모델을 가져와서 여기에 데이터를 넣는다. 
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)  # torch.argmax : softmax 했을때의 값들중 큰것의 라벨값을 가져옴 prediction이라고 보면됨
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)  # 테스트 데이터에도 라벨은 남아있음 Test accuracy:  0.8100405020251013

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Test accuracy:  0.8100405020251013
